In [1]:
!pip install yfinance python-dotenv psycopg2-binary

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 9.4 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 1.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 6.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 18.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.4/159.4 kB 13.1 MB/s eta 0:00:00
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15548 sha256=f7858b59dd782f945932368095913e828dc46e1bbed856f7972be7c773d4a351
  Stored in directory: /home/jovyan/.cache/pip/wheels/42/d6/84/bf57a755f4569494cd00de4bb46ef064874823f4d19c82e960
  Created wheel for peewee: filename=

In [1]:
import os
from datetime import datetime
import sys

import yfinance as yf
import pandas as pd
from dotenv import load_dotenv
import psycopg2
from psycopg2.extras import execute_values

print("✅ Librerías importadas correctamente")

✅ Librerías importadas correctamente


In [2]:
# Cargar variables de entorno
env_path = "/home/jovyan/work/.env"
if os.path.exists(env_path):
    load_dotenv(env_path)
    print(f"✅ .env cargado desde: {env_path}")
else:
    print("⚠️  No se encontró archivo .env. Usando variables del sistema.")

# Leer parámetros de ingesta
tickers_str = os.getenv("TICKERS", "AAPL,MSFT,GOOG")
start_date = os.getenv("START_DATE", "2018-01-01")
end_date = os.getenv("END_DATE", "2025-01-01")
run_id = os.getenv("RUN_ID", "dev_run_1")
data_provider = os.getenv("DATA_PROVIDER", "yfinance")

pg_host = os.getenv("PG_HOST", "postgres")
pg_port = os.getenv("PG_PORT", "5432")
pg_db = os.getenv("PG_DB", "marketdb")
pg_user = os.getenv("PG_USER", "postgres")
pg_password = os.getenv("PG_PASSWORD", "postgres")
pg_schema_raw = os.getenv("PG_SCHEMA_RAW", "raw")
pg_schema_analytics = os.getenv("PG_SCHEMA_ANALYTICS", "analytics")

tickers = [t.strip() for t in tickers_str.split(",") if t.strip()]

print("📊 Parámetros de ingesta:")
print(f"   TICKERS: {tickers}")
print(f"   Rango fechas: {start_date} a {end_date}")
print(f"   RUN_ID: {run_id}")
print(f"   Proveedor: {data_provider}")
print(f"   Esquema RAW: {pg_schema_raw}")
print(f"   Esquema ANALYTICS: {pg_schema_analytics}")

✅ .env cargado desde: /home/jovyan/work/.env
📊 Parámetros de ingesta:
   TICKERS: ['AAPL', 'MSFT', 'GOOG']
   Rango fechas: 2018-01-01 a 2025-01-01
   RUN_ID: dev_run_1
   Proveedor: yfinance
   Esquema RAW: raw
   Esquema ANALYTICS: analytics


In [3]:
def get_connection():
    """
    Crea una conexión a Postgres usando las variables de entorno.
    """
    try:
        conn = psycopg2.connect(
            host=pg_host,
            port=pg_port,
            dbname=pg_db,
            user=pg_user,
            password=pg_password
        )
        print("✅ Conexión a PostgreSQL establecida")
        return conn
    except Exception as e:
        print(f"❌ Error conectando a PostgreSQL: {str(e)}")
        raise

In [4]:
def create_schemas_and_tables():
    """Crear esquemas y tablas necesarias en PostgreSQL"""
    conn = get_connection()
    cur = conn.cursor()
    
    # Crear esquemas
    cur.execute(f"CREATE SCHEMA IF NOT EXISTS {pg_schema_raw};")
    cur.execute(f"CREATE SCHEMA IF NOT EXISTS {pg_schema_analytics};")
    print(f"✅ Esquemas '{pg_schema_raw}' y '{pg_schema_analytics}' creados/verificados")
    
    # Crear tabla raw.prices_daily
    create_raw_table_sql = f"""
    CREATE TABLE IF NOT EXISTS {pg_schema_raw}.prices_daily (
        date DATE NOT NULL,
        ticker TEXT NOT NULL,
        open DOUBLE PRECISION,
        high DOUBLE PRECISION,
        low DOUBLE PRECISION,
        close DOUBLE PRECISION,
        adj_close DOUBLE PRECISION,
        volume BIGINT,
        run_id TEXT,
        ingested_at_utc TIMESTAMP,
        source_name TEXT,
        PRIMARY KEY (date, ticker)
    );
    """
    cur.execute(create_raw_table_sql)
    print(f"✅ Tabla '{pg_schema_raw}.prices_daily' creada/verificada")
    
    # Crear tabla analytics.daily_features (estructura básica)
    create_analytics_table_sql = f"""
    CREATE TABLE IF NOT EXISTS {pg_schema_analytics}.daily_features (
        date DATE NOT NULL,
        ticker TEXT NOT NULL,
        year INTEGER,
        month INTEGER,
        day_of_week INTEGER,
        open DOUBLE PRECISION,
        close DOUBLE PRECISION,
        high DOUBLE PRECISION,
        low DOUBLE PRECISION,
        volume BIGINT,
        return_close_open DOUBLE PRECISION,
        return_prev_close DOUBLE PRECISION,
        run_id TEXT,
        ingested_at_utc TIMESTAMP,
        PRIMARY KEY (date, ticker)
    );
    """
    cur.execute(create_analytics_table_sql)
    print(f"✅ Tabla '{pg_schema_analytics}.daily_features' creada/verificada")
    
    conn.commit()
    cur.close()
    conn.close()

# Ejecutar creación de esquemas y tablas
create_schemas_and_tables()

✅ Conexión a PostgreSQL establecida
✅ Esquemas 'raw' y 'analytics' creados/verificados
✅ Tabla 'raw.prices_daily' creada/verificada
✅ Tabla 'analytics.daily_features' creada/verificada


In [5]:
print("📥 Descargando datos desde Yahoo Finance...")

ingested_at_utc = datetime.utcnow()
records = []

# Manejar el caso de un solo ticker o múltiples tickers
if len(tickers) == 1:
    tk = tickers[0]
    print(f"Descargando datos para {tk}...")
    data = yf.download(
        tickers=tk,
        start=start_date,
        end=end_date,
        auto_adjust=False
    )
    data["ticker"] = tk
    data["date"] = data.index
    data["run_id"] = run_id
    data["ingested_at_utc"] = ingested_at_utc
    data["source_name"] = data_provider
    data = data.reset_index(drop=True)
    records.append(data)
else:
    print(f"Descargando datos para {len(tickers)} tickers: {tickers}")
    data = yf.download(
        tickers=tickers,
        start=start_date,
        end=end_date,
        group_by="ticker",
        auto_adjust=False
    )
    
    for tk in tickers:
        if tk in data:
            print(f"Procesando {tk}...")
            df = data[tk].copy()
            df["ticker"] = tk
            df["date"] = df.index
            df["run_id"] = run_id
            df["ingested_at_utc"] = ingested_at_utc
            df["source_name"] = data_provider
            df = df.reset_index(drop=True)
            records.append(df)
        else:
            print(f"⚠️  No se encontraron datos para {tk}")

if not records:
    raise Exception("❌ No se pudieron descargar datos para ningún ticker")

print("✅ Descarga de Yahoo Finance completada")

📥 Descargando datos desde Yahoo Finance...
Descargando datos para 3 tickers: ['AAPL', 'MSFT', 'GOOG']


[*********************100%***********************]  3 of 3 completed

Procesando AAPL...
Procesando MSFT...
Procesando GOOG...
✅ Descarga de Yahoo Finance completada


In [6]:
print("🔄 Transformando datos a formato 'One Big Table'...")

df_final = pd.concat(records, ignore_index=True)

# Renombrar columnas a snake_case estándar
df_final = df_final.rename(columns={
    "Open": "open",
    "High": "high",
    "Low": "low",
    "Close": "close",
    "Adj Close": "adj_close",
    "Volume": "volume"
})

# Asegurar tipos básicos
df_final["date"] = pd.to_datetime(df_final["date"]).dt.date

print(f"✅ Transformación completada: {len(df_final)} registros")
print("\n📋 Muestra de datos transformados:")
df_final.head()

🔄 Transformando datos a formato 'One Big Table'...
✅ Transformación completada: 5283 registros

📋 Muestra de datos transformados:


Price,open,high,low,close,adj_close,volume,ticker,date,run_id,ingested_at_utc,source_name
0,42.540001,43.075001,42.314999,43.064999,40.341892,102223600,AAPL,2018-01-02,dev_run_1,2025-11-30 16:02:44.919523,yfinance
1,43.132500,43.637501,42.990002,43.057499,40.334854,118071600,AAPL,2018-01-03,dev_run_1,2025-11-30 16:02:44.919523,yfinance
2,43.134998,43.367500,43.020000,43.257500,40.522221,89738400,AAPL,2018-01-04,dev_run_1,2025-11-30 16:02:44.919523,yfinance
3,43.360001,43.842499,43.262501,43.750000,40.983585,94640000,AAPL,2018-01-05,dev_run_1,2025-11-30 16:02:44.919523,yfinance
4,43.587502,43.902500,43.482498,43.587502,40.831360,82271200,AAPL,2018-01-08,dev_run_1,2025-11-30 16:02:44.919523,yfinance


In [7]:
def insert_prices_data(df):
    """Insertar datos en PostgreSQL"""
    conn = get_connection()
    cur = conn.cursor()

    insert_sql = f"""
    INSERT INTO {pg_schema_raw}.prices_daily
    (date, ticker, open, high, low, close, adj_close, volume, run_id, ingested_at_utc, source_name)
    VALUES %s
    ON CONFLICT (date, ticker) DO NOTHING;
    """

    # Convertir DataFrame a lista de tuplas
    records = [
        (
            row["date"],
            row["ticker"],
            float(row["open"]) if pd.notnull(row["open"]) else None,
            float(row["high"]) if pd.notnull(row["high"]) else None,
            float(row["low"]) if pd.notnull(row["low"]) else None,
            float(row["close"]) if pd.notnull(row["close"]) else None,
            float(row["adj_close"]) if pd.notnull(row["adj_close"]) else None,
            int(row["volume"]) if pd.notnull(row["volume"]) else None,
            row["run_id"],
            row["ingested_at_utc"],
            row["source_name"],
        )
        for _, row in df.iterrows()
    ]

    print(f"💾 Insertando {len(records)} registros en {pg_schema_raw}.prices_daily...")
    
    execute_values(cur, insert_sql, records)
    conn.commit()
    
    # Obtener conteo real de inserciones (sin duplicados)
    cur.execute(f"SELECT COUNT(*) FROM {pg_schema_raw}.prices_daily WHERE run_id = %s;", (run_id,))
    inserted_count = cur.fetchone()[0]
    
    cur.close()
    conn.close()
    
    print(f"✅ Inserción completada: {inserted_count} registros insertados")
    return inserted_count

# Ejecutar inserción
inserted_count = insert_prices_data(df_final)

✅ Conexión a PostgreSQL establecida
💾 Insertando 5283 registros en raw.prices_daily...
✅ Inserción completada: 5283 registros insertados


In [8]:
def verify_and_show_results():
    """Verificar y mostrar resultados de la ingesta"""
    conn = get_connection()
    
    # Resumen general
    query_summary = f"""
    SELECT 
        COUNT(*) AS n_rows,
        MIN(date) AS min_date,
        MAX(date) AS max_date,
        COUNT(DISTINCT ticker) AS n_tickers,
        COUNT(DISTINCT date) AS n_dias_bursatiles
    FROM {pg_schema_raw}.prices_daily;
    """
    
    summary = pd.read_sql(query_summary, conn)
    
    print("\n" + "="*50)
    print("📈 RESUMEN FINAL DE INGESTA")
    print("="*50)
    print(f"   • Total registros en BD: {summary['n_rows'].iloc[0]}")
    print(f"   • Fecha mínima: {summary['min_date'].iloc[0]}")
    print(f"   • Fecha máxima: {summary['max_date'].iloc[0]}")
    print(f"   • Número de tickers: {summary['n_tickers'].iloc[0]}")
    print(f"   • Días bursátiles únicos: {summary['n_dias_bursatiles'].iloc[0]}")
    
    # Resumen por ticker
    query_by_ticker = f"""
    SELECT 
        ticker,
        COUNT(*) AS n_registros,
        MIN(date) AS primera_fecha,
        MAX(date) AS ultima_fecha
    FROM {pg_schema_raw}.prices_daily
    GROUP BY ticker
    ORDER BY ticker;
    """
    
    by_ticker = pd.read_sql(query_by_ticker, conn)
    
    print(f"\n📊 DETALLE POR TICKER:")
    for _, row in by_ticker.iterrows():
        print(f"   • {row['ticker']}: {row['n_registros']} registros ({row['primera_fecha']} a {row['ultima_fecha']})")
    
    # Muestra de los últimos registros
    query_sample = f"""
    SELECT *
    FROM {pg_schema_raw}.prices_daily
    ORDER BY date DESC, ticker
    LIMIT 10;
    """
    
    sample = pd.read_sql(query_sample, conn)
    
    print(f"\n🔍 MUESTRA DE REGISTROS MÁS RECIENTES:")
    display(sample)
    
    conn.close()
    
    return summary, by_ticker

# Ejecutar verificación
summary, by_ticker = verify_and_show_results()

✅ Conexión a PostgreSQL establecida

📈 RESUMEN FINAL DE INGESTA
   • Total registros en BD: 5283
   • Fecha mínima: 2018-01-02
   • Fecha máxima: 2024-12-31
   • Número de tickers: 3
   • Días bursátiles únicos: 1761

📊 DETALLE POR TICKER:
   • AAPL: 1761 registros (2018-01-02 a 2024-12-31)
   • GOOG: 1761 registros (2018-01-02 a 2024-12-31)
   • MSFT: 1761 registros (2018-01-02 a 2024-12-31)

🔍 MUESTRA DE REGISTROS MÁS RECIENTES:


/tmp/ipykernel_124/3922237902.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  summary = pd.read_sql(query_summary, conn)
/tmp/ipykernel_124/3922237902.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  by_ticker = pd.read_sql(query_by_ticker, conn)
/tmp/ipykernel_124/3922237902.py:53: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sample = pd.read_sql(query_sample, conn)


,date,ticker,open,high,low,close,adj_close,volume,run_id,ingested_at_utc,source_name
0,2024-12-31,AAPL,252.440002,253.279999,249.429993,250.419998,249.292511,39480700,dev_run_1,2025-11-29 11:17:38.502389,yfinance
1,2024-12-31,GOOG,192.445007,193.250000,189.580002,190.440002,189.825241,14355200,dev_run_1,2025-11-29 11:17:38.502389,yfinance
2,2024-12-31,MSFT,426.100006,426.730011,420.660004,421.500000,418.413452,13246500,dev_run_1,2025-11-29 11:17:38.502389,yfinance
3,2024-12-30,AAPL,252.229996,253.500000,250.750000,252.199997,251.064484,35557500,dev_run_1,2025-11-29 11:17:38.502389,yfinance
4,2024-12-30,GOOG,190.865005,193.779999,190.360001,192.690002,192.067993,12209500,dev_run_1,2025-11-29 11:17:38.502389,yfinance
5,2024-12-30,MSFT,426.059998,427.549988,421.899994,424.829987,421.719086,13158700,dev_run_1,2025-11-29 11:17:38.502389,yfinance
6,2024-12-27,AAPL,257.829987,258.700012,253.059998,255.589996,254.439224,42355300,dev_run_1,2025-11-29 11:17:38.502389,yfinance
7,2024-12-27,GOOG,196.470001,196.800003,191.972000,194.039993,193.413620,14693000,dev_run_1,2025-11-29 11:17:38.502389,yfinance
8,2024-12-27,MSFT,434.600006,435.220001,426.350006,430.529999,427.377319,18117700,dev_run_1,2025-11-29 11:17:38.502389,yfinance
9,2024-12-26,AAPL,258.190002,260.100006,257.630005,259.019989,257.853790,27237100,dev_run_1,2025-11-29 11:17:38.502389,yfinance


In [9]:
def check_temporal_coverage():
    """Verificar cobertura temporal y días sin datos"""
    conn = get_connection()
    
    # Verificar días bursátiles esperados vs reales
    query_coverage = f"""
    WITH expected_dates AS (
        SELECT DISTINCT date
        FROM {pg_schema_raw}.prices_daily
        WHERE ticker = %s
    ),
    all_trading_days AS (
        SELECT generate_series(
            MIN(date)::timestamp, 
            MAX(date)::timestamp, 
            '1 day'::interval
        )::date as trading_date
        FROM {pg_schema_raw}.prices_daily
        WHERE ticker = %s
    ),
    missing_days AS (
        SELECT trading_date
        FROM all_trading_days
        WHERE EXTRACT(DOW FROM trading_date) BETWEEN 1 AND 5  -- Lunes a Viernes
        AND trading_date NOT IN (SELECT date FROM expected_dates)
    )
    SELECT 
        COUNT(*) as total_trading_days,
        (SELECT COUNT(*) FROM expected_dates) as available_days,
        (SELECT COUNT(*) FROM missing_days) as missing_days_count,
        (SELECT ARRAY_AGG(trading_date) FROM missing_days LIMIT 10) as sample_missing_days
    FROM all_trading_days
    WHERE EXTRACT(DOW FROM trading_date) BETWEEN 1 AND 5;
    """
    
    print("\n📅 VERIFICACIÓN DE COBERTURA TEMPORAL:")
    
    for ticker in tickers:
        cur = conn.cursor()
        cur.execute(query_coverage, (ticker, ticker))
        result = cur.fetchone()
        cur.close()
        
        if result:
            total_days, available_days, missing_count, sample_missing = result
            coverage_pct = (available_days / total_days) * 100 if total_days > 0 else 0
            
            print(f"\n   {ticker}:")
            print(f"     • Días bursátiles totales: {total_days}")
            print(f"     • Días con datos: {available_days}")
            print(f"     • Días sin datos: {missing_count}")
            print(f"     • Cobertura: {coverage_pct:.2f}%")
            
            if missing_count > 0:
                print(f"     • Ejemplo días faltantes: {sample_missing[:5] if sample_missing else 'N/A'}")
    
    conn.close()

# Ejecutar verificación de cobertura
check_temporal_coverage()

✅ Conexión a PostgreSQL establecida

📅 VERIFICACIÓN DE COBERTURA TEMPORAL:

   AAPL:
     • Días bursátiles totales: 1826
     • Días con datos: 1761
     • Días sin datos: 65
     • Cobertura: 96.44%
     • Ejemplo días faltantes: [datetime.date(2018, 1, 15), datetime.date(2018, 2, 19), datetime.date(2018, 3, 30), datetime.date(2018, 5, 28), datetime.date(2018, 7, 4)]

   MSFT:
     • Días bursátiles totales: 1826
     • Días con datos: 1761
     • Días sin datos: 65
     • Cobertura: 96.44%
     • Ejemplo días faltantes: [datetime.date(2018, 1, 15), datetime.date(2018, 2, 19), datetime.date(2018, 3, 30), datetime.date(2018, 5, 28), datetime.date(2018, 7, 4)]

   GOOG:
     • Días bursátiles totales: 1826
     • Días con datos: 1761
     • Días sin datos: 65
     • Cobertura: 96.44%
     • Ejemplo días faltantes: [datetime.date(2018, 1, 15), datetime.date(2018, 2, 19), datetime.date(2018, 3, 30), datetime.date(2018, 5, 28), datetime.date(2018, 7, 4)]


In [10]:
print("\n" + "="*60)
print("🎉 INGESTA COMPLETADA EXITOSAMENTE")
print("="*60)
print(f"✅ Datos ingresados en: {pg_schema_raw}.prices_daily")
print(f"✅ Total registros procesados: {len(df_final)}")
print(f"✅ Registros únicos insertados: {inserted_count}")
print(f"✅ Tickers procesados: {tickers}")
print(f"✅ Período cubierto: {summary['min_date'].iloc[0]} a {summary['max_date'].iloc[0]}")
print(f"✅ RUN_ID: {run_id}")
print("\n📝 Próximos pasos:")
print("   1. Ejecutar feature-builder para crear analytics.daily_features")
print("   2. Verificar datos en PostgreSQL con: SELECT * FROM raw.prices_daily LIMIT 10;")
print("="*60)


🎉 INGESTA COMPLETADA EXITOSAMENTE
✅ Datos ingresados en: raw.prices_daily
✅ Total registros procesados: 5283
✅ Registros únicos insertados: 5283
✅ Tickers procesados: ['AAPL', 'MSFT', 'GOOG']
✅ Período cubierto: 2018-01-02 a 2024-12-31
✅ RUN_ID: dev_run_1

📝 Próximos pasos:
   1. Ejecutar feature-builder para crear analytics.daily_features
   2. Verificar datos en PostgreSQL con: SELECT * FROM raw.prices_daily LIMIT 10;


In [11]:
conn = get_connection()
df_summary_analytics = pd.read_sql("""
    SELECT
        COUNT(*) AS n_rows,
        MIN(date) AS min_date,
        MAX(date) AS max_date,
        COUNT(DISTINCT ticker) AS n_tickers
    FROM analytics.daily_features;
""", conn)



✅ Conexión a PostgreSQL establecida


/tmp/ipykernel_124/2836786316.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_summary_analytics = pd.read_sql("""


In [12]:
df_sample_analytics = pd.read_sql("""
    SELECT *
    FROM analytics.daily_features
    WHERE ticker = 'AAPL'
    ORDER BY date DESC
    LIMIT 10;
""", conn)

conn.close()
df_sample_analytics


/tmp/ipykernel_124/134320247.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sample_analytics = pd.read_sql("""


,date,ticker,year,month,day_of_week,open,high,low,close,volume,return_close_open,return_prev_close,volatility_5d,run_id,ingested_at_utc
0,2024-12-31,AAPL,2024,12,1,252.440002,253.279999,249.429993,250.419998,39480700,-0.008002,-0.007058,0.010856,run_full_aapl_1,2025-11-30 14:09:14.019794
1,2024-12-30,AAPL,2024,12,0,252.229996,253.500000,250.750000,252.199997,35557500,-0.000119,-0.013263,0.011035,run_full_aapl_1,2025-11-30 14:09:14.019794
2,2024-12-27,AAPL,2024,12,4,257.829987,258.700012,253.059998,255.589996,42355300,-0.008688,-0.013242,0.011959,run_full_aapl_1,2025-11-30 14:09:14.019794
3,2024-12-26,AAPL,2024,12,3,258.190002,260.100006,257.630005,259.019989,27237100,0.003215,0.003176,0.006617,run_full_aapl_1,2025-11-30 14:09:14.019794
4,2024-12-24,AAPL,2024,12,1,255.490005,258.209991,255.289993,258.200012,23234700,0.010607,0.011478,0.015260,run_full_aapl_1,2025-11-30 14:09:14.019794
5,2024-12-23,AAPL,2024,12,0,254.770004,255.649994,253.449997,255.270004,40858800,0.001963,0.003065,0.015057,run_full_aapl_1,2025-11-30 14:09:14.019794
6,2024-12-20,AAPL,2024,12,4,248.039993,255.000000,245.690002,254.490005,147495300,0.026004,0.018816,0.015495,run_full_aapl_1,2025-11-30 14:09:14.019794
7,2024-12-19,AAPL,2024,12,3,247.500000,252.000000,247.089996,249.789993,60882300,0.009252,0.007015,0.013497,run_full_aapl_1,2025-11-30 14:09:14.019794
8,2024-12-18,AAPL,2024,12,2,252.160004,254.279999,247.740005,248.050003,56774100,-0.016299,-0.021422,0.013398,run_full_aapl_1,2025-11-30 14:09:14.019794
9,2024-12-17,AAPL,2024,12,1,250.080002,253.830002,249.779999,253.479996,51356400,0.013596,0.009720,0.006884,run_full_aapl_1,2025-11-30 14:09:14.019794


NameError: name 'X_test' is not defined